In [1]:
data_var = '2024-01-15'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9701,2024-01-15,Brasil Nbb,19:30,Pato,R10 Score Vasco da Gama,2.10,1.68,148.5,1.86,1.80,1.5,1.95,1.76,OKMPndAs,0.476190,0.595238,0.537634,0.555556,0.071429,0.6,0.6,NaN,NaN,213.728,189.897028,0.888499,189.768,92.947506,0.489795,221.762,198.104,549.99,168.36,1.0,0.0,0.0,0.0,0.157135,0.023184,0.072426,3.56,0.712,1.544944,0.000000,0.0,0.000000,-0.35,-0.070,-9.714286,0.000000,0.0,0.000000
9702,2024-01-15,Brasil Nbb,19:30,São José,Pinheiros,1.30,3.31,154.5,1.87,1.79,-8.5,2.00,1.72,IsMToGPm,0.769231,0.302115,0.534759,0.558659,0.071346,0.6,0.4,NaN,NaN,161.632,39.964628,0.247257,157.286,91.733857,0.583230,122.444,160.434,100.11,132.24,0.0,0.0,0.0,0.0,0.616609,0.030912,0.106446,-0.67,-0.134,-2.238806,0.606771,0.6,-0.006771,-1.59,-0.318,-7.264151,0.000000,0.0,0.000000
9703,2024-01-15,Eua Nba,15:00,Philadelphia 76ers,Houston Rockets,1.40,3.35,222.5,1.95,1.99,-8.5,2.06,1.73,UcZYyvl8,0.714286,0.298507,0.512821,0.502513,0.012793,0.2,0.6,NaN,NaN,183.270,80.337839,0.438358,425.136,405.141975,0.952970,153.394,453.684,213.92,1141.30,0.0,1.0,0.0,0.0,0.580572,0.014357,0.123137,-0.56,-0.112,-3.571429,0.767130,0.7,-0.067130,-0.53,-0.106,-22.169811,0.378799,0.4,0.021201
9704,2024-01-15,Eua Nba,16:30,Dallas Mavericks,New Orleans Pelicans,2.12,1.84,233.5,1.84,1.98,1.5,1.94,1.84,8lYxyb3E,0.471698,0.543478,0.543478,0.505051,0.015176,0.2,0.4,NaN,NaN,187.542,51.431365,0.274239,286.772,73.723201,0.257079,193.222,294.852,212.76,232.46,0.0,0.0,0.0,0.0,0.099995,0.051830,0.037413,-0.34,-0.068,-16.470588,0.608170,0.6,-0.008170,4.23,0.846,0.992908,0.552143,0.7,0.147857
9705,2024-01-15,Eua Nba,17:00,New York Knicks,Orlando Magic,1.39,3.41,221.5,1.85,1.91,-8.5,2.08,1.68,xOxszIIK,0.719424,0.293255,0.540541,0.523560,0.012680,0.4,0.6,NaN,NaN,199.282,113.065734,0.567366,411.326,233.145098,0.566813,215.516,440.476,132.16,624.00,0.0,1.0,0.0,0.0,0.595148,0.022567,0.150448,2.02,0.404,0.965347,0.661759,0.8,0.138241,0.78,0.156,15.448718,0.361383,0.3,-0.061383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9773,2024-01-15,Argentina Liga A,22:00,La Union,Quimsa,3.53,1.27,160.5,1.85,1.85,5.5,2.10,1.67,McXPfVxt,0.283286,0.787402,0.540541,0.540541,0.070688,0.4,0.4,NaN,NaN,164.348,57.676961,0.350944,124.918,19.092654,0.152841,153.204,127.738,117.45,151.20,0.0,0.0,0.0,0.0,0.665859,0.000000,0.161303,-1.88,-0.376,-6.728723,0.000000,0.0,0.000000,2.85,0.570,0.473684,0.000000,0.0,0.000000
9774,2024-01-15,Chipre Divisão A,14:30,AEL,Apollon,1.20,4.22,161.5,1.83,1.87,-11.5,1.95,1.73,xU3M90HJ,0.833333,0.236967,0.546448,0.534759,0.070300,0.0,0.0,NaN,NaN,114.750,30.514395,0.265921,145.114,52.758480,0.363566,0.000,0.000,142.08,145.36,0.0,0.0,0.0,0.0,0.787994,0.015289,0.084545,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9775,2024-01-15,Grécia Liga De Basquete,15:15,Olympiakos,Kolossos Rhodes,1.03,10.50,158.5,1.86,1.88,-22.5,1.94,1.77,x0OlcWgq,0.970874,0.095238,0.537634,0.531915,0.066112,0.6,0.8,NaN,NaN,90.304,6.129799,0.067880,222.196,95.718415,0.430784,87.498,242.930,90.10,337.26,0.0,0.0,0.0,0.0,1.161544,0.007563,0.064802,-0.78,-0.156,-0.192308,0.000000,0.0,0.000000,-2.85,-0.570,-16.666667,0.000000,0.0,0.000000
9776,2024-01-15,Romênia Divisão A,13:00,CSM Targu Mures,SCM Craiova,1.32,3.20,157.5,1.82,1.88,-8.5,2.01,1.68,h0tlR0a3,0.7

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,17:30,Eua Nba,Atlanta Hawks,San Antonio Spurs,244.5,1.83,1.0000,Over
1,21:00,Eua Nba,Cleveland Cavaliers,Chicago Bulls,218.5,1.88,1.0000,Over
2,21:30,Eua Nba,Toronto Raptors,Boston Celtics,239.5,1.85,1.0000,Over
3,23:00,Eua Nba,Utah Jazz,Indiana Pacers,248.5,1.86,1.0000,Over
4,21:00,Eua Ncaa,Le Moyne,Sacred Heart,144.5,1.91,0.8473,Over
